In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('../udataset/wealth_data.csv')
unemployment_data_df = pd.read_csv('../udataset/unemployment_by_state.csv')
health_data_df = pd.read_csv('../udataset/health_data.csv')
commodity_data_df = pd.read_csv('../udataset/all_commodities.csv')

health_insurance_df = pd.read_csv('../udataset/acs/HEALTH INSURANCE COVERAGE.csv')
occupation_df = pd.read_csv('../udataset/acs/OCCUPATION.csv')
industry_df = pd.read_csv('../udataset/acs/INDUSTRY.csv')
employment_df = pd.read_csv('../udataset/acs/EMPLOYMENT STATUS.csv')
commuting_df = pd.read_csv('../udataset/acs/COMMUTING TO WORK.csv')
class_df = pd.read_csv('../udataset/acs/CLASS OF WORKER.csv')

def combine_duplicate_cols(df):
    return df.groupby(level=0, axis=1).sum(min_count=1)

unique_states = class_df['state'].unique()
unique_years = class_df['year'].unique()

pivot_class_data = class_df.pivot_table(index=['state', 'year'], columns='label', values='estimate', aggfunc='first')
pivot_class_data.reset_index(inplace=True)
pivot_class_data.columns= [col.strip() for col in pivot_class_data.columns]
pivot_class_data = combine_duplicate_cols(pivot_class_data)


meat_df = pd.read_csv('../udataset/meat/production.csv')
meat_df = meat_df[meat_df['DateTime'].str.contains(r'-12')]
meat_df = meat_df.rename(columns={'DateTime': 'year'})
meat_df['year'] = meat_df['year'].str.slice(0, 4)
meat_df['year'] = meat_df['year'].astype(int)


unemployment_data_df = unemployment_data_df.rename(columns={'State': 'state'})


unemployment_data_df = unemployment_data_df[unemployment_data_df['YearMonth'].str.contains(r'-12')]
unemployment_data_df = unemployment_data_df.rename(columns={'YearMonth': 'year'})
unemployment_data_df['year'] = unemployment_data_df['year'].str.slice(0, 4)
unemployment_data_df['year'] = unemployment_data_df['year'].astype(int)


health_data_df['state'] = health_data_df['state'].str.lower()

commodity_data_df.drop(columns=['date'], inplace=True)

df = df.merge(meat_df, on=['year'], how='left')


df = df.drop(columns=['Unnamed: 0'])
df = df[df['year'] > 2010]
print(df.shape[1])

model = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

df = df.drop(columns = ['median_household_income', 'mean cash public assistance income (dollars)'])
pivot_class_data



14


/var/folders/_z/09vrqlq533g4mwf_3hh2dzrc0000gn/T/ipykernel_50584/2747490871.py:14: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  return df.groupby(level=0, axis=1).sum(min_count=1)


Civilian employed population 16 years and over  Government workers  \
0                                         2036867.0            332613.0   
1                                         2027919.0            337259.0   
2                                         2017887.0            338248.0   
3                                         2002163.0            337412.0   
4                                         2010453.0            335663.0   
..                                              ...                 ...   
671                                        290963.0             59143.0   
672                                        288503.0             58801.0   
673                                        287375.0             59293.0   
674                                        284934.0             59995.0   
675                                        287895.0             61138.0   

     Private wage and salary workers  \
0                          1577910.0   
1                          1570274.0   
2                          1563062.0   
3                          1550625.0   
4                          1563081.0   
..                               ...   
671                         212611.0   
672                         209833.0   
673                         207858.0   
674                         204810.0   
675                         205909.0   

     Self-employed in own not incorporated business workers  \
0                                             122738.0        
1                                             117230.0        
2                                             113574.0        
3                                             110928.0        
4                                             108300.0        
..                                                 ...        
671                                            18414.0        
672                                            19024.0        
673                                            19108.0        
674                                            19072.0        
675                                            19758.0        

     Unpaid family workers    state  year  
0                   3606.0  Alabama  2010  
1                   3156.0  Alabama  2011  
2                   3003.0  Alabama  2012  
3                   3198.0  Alabama  2013  
4                   3409.0  Alabama  2014  
..                     ...      ...   ...  
671                  795.0  Wyoming  2018  
672                  845.0  Wyoming  2019  
673                 1116.0  Wyoming  2020  
674                 1057.0  Wyoming  2021  
675                 1090.0  Wyoming  2022  

[676 rows x 7 columns]

In [333]:
health_data_df

state  year  obesity_adult_rate  overweight_adult_rate  \
0    alaska  2011            0.346154                   0.98   
1    alaska  2012            0.264423                   1.00   
2    alaska  2013            0.394231                   0.86   
3    alaska  2014            0.456731                   0.60   
4    alaska  2015            0.461538                   0.83   
..      ...   ...                 ...                    ...   
643  oregon  2018            0.466346                   0.51   
644  oregon  2019            0.423077                   0.58   
645  oregon  2020            0.379808                   0.71   
646  oregon  2021            0.490385                   0.56   
647  oregon  2022            0.514423                   0.69   

     adults_no_leisure-time_physical_activity  
0                                    0.177378  
1                                    0.087404  
2                                    0.185090  
3                                    0.105398  
4                                    0.177378  
..                                        ...  
643                                  0.107969  
644                                  0.226221  
645                                  0.087404  
646                                  0.123393  
647                                  0.100257  

[648 rows x 5 columns]

In [334]:
train_df = df[df['year'] < 2021]
test_df = df[df['year'] >= 2021]

train_data = train_df.to_numpy()
train_labels = train_data[:, 2]
test_data = test_df.to_numpy()
test_labels = test_data[:, 2]

train_df = train_df.drop(columns = ['below_poverty_line_percent'])
test_df = test_df.drop(columns = ['below_poverty_line_percent'])
train_df = train_df.drop(columns=['year'])
test_df = test_df.drop(columns=['year'])

train_data = train_df.to_numpy()
test_data = test_df.to_numpy()
train_data[:, 0]
train_data.shape
train_labels.shape

(520,)

In [335]:
le = LabelEncoder()
le.fit(train_data[:, 0])
train_data[:, 0] = le.transform(train_data[:, 0])
train_data.shape

(520, 10)

In [336]:
# define model evaluation method
# evaluate model
# scores = cross_val_score(model, train_data, train_labels, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
model.fit(train_data, train_labels)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, ...)

In [337]:
test_data[:, 0] = le.transform(test_data[:, 0])
test_data.shape

(104, 10)

In [338]:
#scores = absolute(scores)
#print(scores)
#print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )
preds = model.predict(test_data)

In [339]:
preds

array([16.02359  , 16.478113 , 10.322077 , 10.1623335, 14.138607 ,
       14.897432 , 16.13221  , 16.575851 , 12.690953 , 13.300233 ,
        9.858011 , 10.085362 ,  9.762084 ,  9.616054 , 11.364336 ,
       11.35007  , 15.41688  , 15.622983 , 13.36977  , 13.7915   ,
       14.34062  , 14.694039 ,  9.31286  ,  9.219703 , 11.960669 ,
       12.629886 , 11.974027 , 12.131207 , 12.871712 , 13.053467 ,
       11.111572 , 11.35293  , 11.401495 , 11.757939 , 16.591597 ,
       16.911642 , 18.52287  , 18.578419 , 11.115273 , 11.598134 ,
        8.977172 ,  9.041928 ,  9.8023   , 10.1279   , 13.707201 ,
       14.160488 ,  9.321163 ,  9.288968 , 19.547709 , 20.042183 ,
       13.028941 , 13.409514 , 12.8237705, 12.976903 , 10.390802 ,
       10.767436 , 12.919694 , 13.162259 ,  7.4097233,  7.423008 ,
        9.727894 ,  9.786519 , 18.65346  , 19.030828 , 13.995602 ,
       14.009174 , 14.423378 , 14.597873 , 10.855007 , 10.279365 ,
       14.005012 , 13.938292 , 15.628136 , 15.563131 , 12.8629

In [340]:
df

state  year  below_poverty_line_percent          Beef        Veal  \
1    alabama  2011                        17.6  2.096400e+09  10500000.0   
2    alabama  2012                        18.1  1.992300e+09   9600000.0   
3    alabama  2013                        18.6  2.019100e+09   9700000.0   
4    alabama  2014                        18.9  1.973400e+09   7500000.0   
5    alabama  2015                        18.8  2.020500e+09   7700000.0   
..       ...   ...                         ...           ...         ...   
671  wyoming  2018                        11.1  2.091400e+09   6300000.0   
672  wyoming  2019                        11.0  2.239200e+09   6400000.0   
673  wyoming  2020                        10.8  2.291800e+09   5500000.0   
674  wyoming  2021                        10.7  2.326800e+09   5100000.0   
675  wyoming  2022                        10.7  2.193700e+09   4400000.0   

             Pork  Lamb and Mutton       Broiler  Other Chicken       Turkey  \
1    2.052600e+09       11800000.0  2.857500e+09     41400000.0  511400000.0   
2    1.940500e+09       11900000.0  2.853700e+09     36300000.0  515300000.0   
3    2.053900e+09       12300000.0  3.093300e+09     40100000.0  479300000.0   
4    2.103100e+09       12600000.0  3.348500e+09     44300000.0  480800000.0   
5    2.194500e+09       12300000.0  3.334500e+09     44200000.0  470700000.0   
..            ...              ...           ...            ...          ...   
671  2.221100e+09       11900000.0  3.228700e+09     36800000.0  497400000.0   
672  2.432800e+09       11000000.0  3.647300e+09     41800000.0  469200000.0   
673  2.499300e+09       10700000.0  3.720200e+09     42500000.0  458100000.0   
674  2.393800e+09       10500000.0  3.700100e+09     44700000.0  453300000.0   
675  2.220400e+09        9800000.0  3.851600e+09     46000000.0  428200000.0   

         Red Meat       Poultry  
1    4.171300e+09  3.410300e+09  
2    3.954300e+09  3.405300e+09  
3    4.095000e+09  3.612700e+09  
4    4.096600e+09  3.873600e+09  
5    4.235000e+09  3.849400e+09  
..            ...           ...  
671  4.330700e+09  3.762900e+09  
672  4.689400e+09  4.158300e+09  
673  4.807300e+09  4.220800e+09  
674  4.736200e+09  4.198100e+09  
675  4.428300e+09  4.325800e+09  

[624 rows x 12 columns]

In [341]:
(absolute(test_labels - preds)).mean()


0.49156132477980385

In [344]:
print(len(preds))
x = np.array(range(len(preds)))
y1 = test_labels
y2 = preds
plt.plot(x, y1)
plt.plot(x, y2)

104
